In [85]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\vinyj\AppData\Roaming\Python\Python313\site-packages\IPython\core\display.py)

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [90]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [88]:
display(data)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


In [18]:
data["label"].value_counts()

label
0    558
1    442
Name: count, dtype: int64

### Let's divide the training and test set into two partitions

In [91]:
from sklearn.model_selection import train_test_split

X = data["text"]
y = data["label"].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Data Preprocessing

In [92]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
from bs4 import BeautifulSoup

def clean_html(text):
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Apply the cleaning function to the text data
data['clean_text'] = data['text'].apply(clean_html)


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [50]:
import re

def preprocess_text(text):
    # Remove tudo que não for letra
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Remove caracteres únicos
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove caracteres únicos no início
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    # Substitui múltiplos espaços
    text = re.sub(r'\s+', ' ', text)
    # Converte para minúsculo
    text = text.lower()
    return text

data['clean_text'] = data['clean_text'].apply(preprocess_text)


## Now let's work on removing stopwords
Remove the stopwords.

In [93]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinyj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
stop_words = set(stopwords.words('english'))
print(list(stop_words)[:20])  # show first 20 stopwords

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

['shan', 'how', 'the', "hasn't", 'here', "we'll", 'needn', 'haven', 'himself', "i've", 't', 'those', 'against', 'can', 'having', 'few', 'aren', 'is', 'have', 'about']


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [95]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    words = nltk.word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


In [96]:
sample_text = "Cats are running faster than the dogs"
print(clean_text(sample_text))

cat running faster dog


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np  

vectorizer = CountVectorizer(lowercase=True, token_pattern=r"[a-zA-Z]+", min_df=2)

X_bow = vectorizer.fit_transform(X_test)

# Mostrar shape da matriz (docs x features)
print("BoW Shape:", X_bow.shape)

# Mostrar algumas features
print("First 20 features:", vectorizer.get_feature_names_out()[:20])

# Top palavras globais
word_counts = np.asarray(X_bow.sum(axis=0)).ravel()
top_idx = word_counts.argsort()[::-1][:20]
top_words = [(vectorizer.get_feature_names_out()[i], int(word_counts[i])) for i in top_idx]

print("\nTop 20 words at corpus:")
for w,c in top_words:
    print(f"{w}: {c}")


BoW Shape: (200, 1977)
First 20 features: ['a' 'aamsz' 'abacha' 'abandoned' 'abedinh' 'abidjan' 'ability' 'able'
 'about' 'above' 'abroad' 'absolute' 'absolutely' 'accept' 'acceptable'
 'acceptance' 'access' 'accident' 'according' 'accordingly']

Top 20 words at corpus:
the: 1499
to: 1075
of: 988
and: 746
in: 679
i: 606
you: 577
a: 551
br: 504
this: 476
e: 420
for: 364
my: 354
that: 346
your: 338
will: 315
is: 312
as: 307
with: 268
be: 255


## Extra features

In [105]:
data_train = data.copy()

In [106]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

KeyError: 'preprocessed_text'

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer 

mask_nonempty = data['text'].str.strip().astype(bool)
X_test = data.loc[mask_nonempty, 'text']
y = data.loc[mask_nonempty, 'label']

Tfidf = TfidfVectorizer(lowercase=True, token_pattern=r"[a-zA-Z]+", min_df=2)

X_tfidf = Tfidf.fit_transform(X_test)

print("TF-IDF Shape:", X_tfidf.shape)
print("First 20 features:", Tfidf.get_feature_names_out()[:20])

TF-IDF Shape: (999, 7133)
First 20 features: ['a' 'aa' 'aaa' 'aacute' 'aamsz' 'ab' 'aba' 'abacha' 'abad' 'abandon'
 'abandoned' 'abank' 'abbas' 'abbass' 'abdul' 'abdullah' 'abedin'
 'abedinh' 'abedinre' 'aberrational']


## And the Train a Classifier?

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Training Model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluation
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.965

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       112
           1       0.99      0.93      0.96        88

    accuracy                           0.96       200
   macro avg       0.97      0.96      0.96       200
weighted avg       0.97      0.96      0.96       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code